In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15060153
paper_name = 'lawrence_coote_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 99 x 3


In [7]:
original_data.head()

,Functional category and ORF,Gene,Characteristic or description
0,Metabolism,NaN,NaN
1,"Lipid, fatty acid, and isoprenoid metabolism",NaN,NaN
2,YER019W,ISC1,Weakly similar to human and mouse neutral sphi...
3,YMR207c,HFA1,Strongly similar to acetyl coenzyme A carboxylase
4,YMR202w,ERG2,C8 sterol isomerase


In [8]:
original_data['orf'] = original_data['Functional category and ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                   Functional category and ORF Gene  \
index_input                                                           
0                                                   Metabolism  NaN   
1                 Lipid, fatty acid, and isoprenoid metabolism  NaN   
7                                      Carbohydrate metabolism  NaN   
11                                       Nucleotide metabolism  NaN   
13                                       Amino acid metabolism  NaN   
19                               Metabolism of energy reserves  NaN   
21                                Energy generation activities  NaN   
23           Regulation of interaction with cellular enviro...  NaN   
24                                             Ion homeostasis  NaN   
36                 Cellular transport and transport mechanisms  NaN   
38                               Control cellular organization  NaN   
39                                                   Cell wall  NaN   
46    

In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data['data'] = -1

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(78, 1)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [190]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,190
data_type,value
orf,
YAL016W,-1
YAL026C,-1
YBL058W,-1
YBR101C,-1
YBR112C,-1


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,190
,data_type,value
gene_id,orf,
14,YAL016W,-1
24,YAL026C,-1
146,YBL058W,-1
295,YBR101C,-1
306,YBR112C,-1


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        190          
data_type       value    valuez
gene_id orf                    
14      YAL016W    -1 -7.645176
24      YAL026C    -1 -7.645176
146     YBL058W    -1 -7.645176
295     YBR101C    -1 -7.645176
306     YBR112C    -1 -7.645176

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15060153...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.53it/s]

Updating the data_modified_on field...
